In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from datasets import load_dataset

/home/alexpy/Desktop/poridhi-ai-hackathon/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

# Initialize the Spark session
spark = SparkSession.builder \
    .appName("Fashion Dataset Cleaning") \
    .getOrCreate()

25/04/26 15:30:30 WARN Utils: Your hostname, indrani resolves to a loopback address: 127.0.1.1; using 10.112.62.61 instead (on interface wlo1)
25/04/26 15:30:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/26 15:30:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
ds = load_dataset("wdc/products-2017", "cameras_large")

In [7]:
pandas_df = ds["train"].to_pandas()

In [8]:
spark_df = spark.createDataFrame(pandas_df)

In [7]:
spark_df.count()

25/04/26 14:44:04 WARN TaskSetManager: Stage 0 contains a task of very large size (5395 KiB). The maximum recommended task size is 1000 KiB.


16028

In [13]:
from pyspark.sql import functions as F

# Assuming 'spark_df' is your PySpark DataFrame
null_count = spark_df.select([F.sum(F.col(c).isNull().cast("int")).alias(c) for c in spark_df.columns])

# Show the result
null_count.show()

25/04/26 12:51:10 WARN TaskSetManager: Stage 2 contains a task of very large size (5395 KiB). The maximum recommended task size is 1000 KiB.


+-------+-----+-------+-------------+---------------+----------+----------+----------------+----------+---------------------+--------+--------------+----------------+-----------+-----------+-----------------+-----------+----------------------+
|pair_id|label|id_left|category_left|cluster_id_left|brand_left|title_left|description_left|price_left|specTableContent_left|id_right|category_right|cluster_id_right|brand_right|title_right|description_right|price_right|specTableContent_right|
+-------+-----+-------+-------------+---------------+----------+----------+----------------+----------+---------------------+--------+--------------+----------------+-----------+-----------+-----------------+-----------+----------------------+
|      0|    0|      0|            0|              0|      8851|         0|            3337|     14405|                13728|       0|             0|               0|       9179|          0|             3774|      14643|                 13555|
+-------+-----+-------+-

In [9]:
filtered_df = spark_df.filter(
    spark_df['price_left'].isNotNull() & spark_df['price_right'].isNotNull()
)

# Take the first 1000 rows
result_df = filtered_df.limit(1000)

In [10]:
result_df.show(5)

25/04/26 15:34:21 WARN TaskSetManager: Stage 0 contains a task of very large size (5395 KiB). The maximum recommended task size is 1000 KiB.


+-----------------+-----+--------+----------------+---------------+----------+--------------------+--------------------+---------------+---------------------+--------+----------------+----------------+-----------+--------------------+--------------------+---------------+----------------------+
|          pair_id|label| id_left|   category_left|cluster_id_left|brand_left|          title_left|    description_left|     price_left|specTableContent_left|id_right|  category_right|cluster_id_right|brand_right|         title_right|   description_right|    price_right|specTableContent_right|
+-----------------+-----+--------+----------------+---------------+----------+--------------------+--------------------+---------------+---------------------+--------+----------------+----------------+-----------+--------------------+--------------------+---------------+----------------------+
| 15548233#6849009|    1|15548233|Camera_and_Photo|         220397|     "IBM"| "IBM E445748 ABT...| "E445748 ABT 8M

In [11]:
result_df = result_df.drop("pair_id", "label", "specTableContent_left", "specTableContent_right")
result_df.printSchema()

root
 |-- id_left: long (nullable = true)
 |-- category_left: string (nullable = true)
 |-- cluster_id_left: long (nullable = true)
 |-- brand_left: string (nullable = true)
 |-- title_left: string (nullable = true)
 |-- description_left: string (nullable = true)
 |-- price_left: string (nullable = true)
 |-- id_right: long (nullable = true)
 |-- category_right: string (nullable = true)
 |-- cluster_id_right: long (nullable = true)
 |-- brand_right: string (nullable = true)
 |-- title_right: string (nullable = true)
 |-- description_right: string (nullable = true)
 |-- price_right: string (nullable = true)



In [12]:
result_df.write.json("output.json", mode="overwrite")

25/04/26 15:34:25 WARN TaskSetManager: Stage 1 contains a task of very large size (5395 KiB). The maximum recommended task size is 1000 KiB.


In [13]:
spark.stop()

## Remove missing data

In [14]:
df_cleaned = df.dropna(how='any')

## Drop Duplicates

In [15]:
df_cleaned = df_cleaned.dropDuplicates()

In [16]:
df_cleaned.count()

25/04/16 01:03:26 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , BrandName, Deatils, Sizes, MRP, SellPrice, Discount, Category
 Schema: _c0, BrandName, Deatils, Sizes, MRP, SellPrice, Discount, Category
Expected: _c0 but found: 
CSV file: file:///home/alexpy/Desktop/poridhi.io-ai-hackathon/data_processor/FashionDataset.csv


7014

## Distict values

In [17]:
distinct_counts = df_cleaned.groupBy("Category").count()
distinct_counts.show()

25/04/16 01:03:34 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , BrandName, Deatils, Sizes, MRP, SellPrice, Discount, Category
 Schema: _c0, BrandName, Deatils, Sizes, MRP, SellPrice, Discount, Category
Expected: _c0 but found: 
CSV file: file:///home/alexpy/Desktop/poridhi.io-ai-hackathon/data_processor/FashionDataset.csv


+--------------------+-----+
|            Category|count|
+--------------------+-----+
|      Footwear-Women|  834|
|Lingerie&Nightwea...| 1133|
|   Westernwear-Women| 1696|
|     Fragrance-Women|  289|
|     Jewellery-Women|  635|
|       Watches-Women|  750|
|    Indianwear-Women| 1677|
+--------------------+-----+



## Stop spark

In [18]:
df_cleaned.show(50, False)

25/04/16 01:03:43 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , BrandName, Deatils, Sizes, MRP, SellPrice, Discount, Category
 Schema: _c0, BrandName, Deatils, Sizes, MRP, SellPrice, Discount, Category
Expected: _c0 but found: 
CSV file: file:///home/alexpy/Desktop/poridhi.io-ai-hackathon/data_processor/FashionDataset.csv


+---+-----------------+---------------------------------------------------------------------+----------------------------------------------------------------+---+---------+--------+------------------------+
|_c0|BrandName        |Deatils                                                              |Sizes                                                           |MRP|SellPrice|Discount|Category                |
+---+-----------------+---------------------------------------------------------------------+----------------------------------------------------------------+---+---------+--------+------------------------+
|24 |levis            |skinny fit regular length cotton womens jeans - dk indigo            |Size:28,30,32,34                                                |Nan|3699     |Nan     |Westernwear-Women       |
|18 |campus sutra     |solid cotton hood womens co-ordinate set - pink                      |Size:Large,Medium,Small,X-Large                                 |Nan|1599     |

In [19]:
df_cleaned = df_cleaned.drop("_c0", "MRP", "Discount")

In [20]:
df_cleaned = df_cleaned.withColumnRenamed("Deatils", "Details")

In [12]:
df_cleaned.printSchema()

root
 |-- BrandName: string (nullable = true)
 |-- Details: string (nullable = true)
 |-- Sizes: string (nullable = true)
 |-- SellPrice: string (nullable = true)
 |-- Category: string (nullable = true)



In [21]:
df_converted = df_cleaned.withColumn("SellPrice", col("SellPrice").cast("int"))

In [22]:
df = df_converted

In [26]:
from pyspark.sql.functions import col, isnan
df = df.filter(~isnan(col("Sizes")))
df.show(50, False)

25/04/16 01:07:46 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , BrandName, Deatils, Sizes, MRP, SellPrice, Discount, Category
 Schema: _c0, BrandName, Deatils, Sizes, MRP, SellPrice, Discount, Category
Expected: _c0 but found: 
CSV file: file:///home/alexpy/Desktop/poridhi.io-ai-hackathon/data_processor/FashionDataset.csv


+-----------------------+------------------------------------------------------------------------------------------------+----------------------------------------------------------------+---------+------------------------+
|BrandName              |Details                                                                                         |Sizes                                                           |SellPrice|Category                |
+-----------------------+------------------------------------------------------------------------------------------------+----------------------------------------------------------------+---------+------------------------+
|levis                  |skinny fit regular length cotton womens jeans - dk indigo                                       |Size:28,30,32,34                                                |3699     |Westernwear-Women       |
|campus sutra           |solid cotton hood womens co-ordinate set - pink                                    

In [27]:
# Filter out rows where 'category' is None or null
df = df.filter(col("Category").isNotNull())


In [61]:
df.count()

25/04/16 00:36:56 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , BrandName, Deatils, Sizes, MRP, SellPrice, Discount, Category
 Schema: _c0, BrandName, Deatils, Sizes, MRP, SellPrice, Discount, Category
Expected: _c0 but found: 
CSV file: file:///home/alexpy/Desktop/poridhi.io-ai-hackathon/data_processor/FashionDataset.csv


7014

In [28]:

# Get the total number of records
total_records = df.count()

# Number of records you want to pick
sample_size = 1000

# Get the count of records per category
category_counts = df.groupBy("Category").count().collect()


25/04/16 01:08:02 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , BrandName, Deatils, Sizes, MRP, SellPrice, Discount, Category
 Schema: _c0, BrandName, Deatils, Sizes, MRP, SellPrice, Discount, Category
Expected: _c0 but found: 
CSV file: file:///home/alexpy/Desktop/poridhi.io-ai-hackathon/data_processor/FashionDataset.csv
25/04/16 01:08:03 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , BrandName, Deatils, Sizes, MRP, SellPrice, Discount, Category
 Schema: _c0, BrandName, Deatils, Sizes, MRP, SellPrice, Discount, Category
Expected: _c0 but found: 
CSV file: file:///home/alexpy/Desktop/poridhi.io-ai-hackathon/data_processor/FashionDataset.csv


In [29]:
# Calculate the fraction for each category based on its proportion in the total dataset
fractions = {}
for row in category_counts:
    category = row['Category']
    count = row['count']
    # Calculate the fraction based on the category size
    fractions[category] = (count / total_records) * sample_size / count

# Perform stratified sampling
df_sampled = df.sampleBy("Category", fractions, seed=42)

In [65]:
df_sampled.count()

25/04/16 00:37:51 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , BrandName, Deatils, Sizes, MRP, SellPrice, Discount, Category
 Schema: _c0, BrandName, Deatils, Sizes, MRP, SellPrice, Discount, Category
Expected: _c0 but found: 
CSV file: file:///home/alexpy/Desktop/poridhi.io-ai-hackathon/data_processor/FashionDataset.csv


1049

In [30]:
df_sampled.write.json("output", mode="overwrite")

25/04/16 01:08:11 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , BrandName, Deatils, Sizes, MRP, SellPrice, Discount, Category
 Schema: _c0, BrandName, Deatils, Sizes, MRP, SellPrice, Discount, Category
Expected: _c0 but found: 
CSV file: file:///home/alexpy/Desktop/poridhi.io-ai-hackathon/data_processor/FashionDataset.csv


In [70]:
df_sampled.groupBy("Category").count().show()

25/04/16 00:47:34 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , BrandName, Deatils, Sizes, MRP, SellPrice, Discount, Category
 Schema: _c0, BrandName, Deatils, Sizes, MRP, SellPrice, Discount, Category
Expected: _c0 but found: 
CSV file: file:///home/alexpy/Desktop/poridhi.io-ai-hackathon/data_processor/FashionDataset.csv


+--------------------+-----+
|            Category|count|
+--------------------+-----+
|      Footwear-Women|  119|
|Lingerie&Nightwea...|  168|
|   Westernwear-Women|  264|
|     Fragrance-Women|   44|
|     Jewellery-Women|   86|
|       Watches-Women|  111|
|    Indianwear-Women|  257|
+--------------------+-----+



In [31]:
spark.stop()